MCPify your AWS Lambda with Gateway IAM role Inbound

In [1]:
# Set AWS credentials if not using Amazon SageMaker notebook
import os

# os.environ['AWS_ACCESS_KEY_ID']=''
# os.environ['AWS_SECRET_ACCESS_KEY']=''
os.environ['AWS_DEFAULT_REGION']='ap-southeast-2' # set the AWS region

In [ ]:
# import os
# import sys

# # Get the directory of the current script
# if '__file__' in globals():
#     current_dir = os.path.dirname(os.path.abspath(__file__))
# else:
#     current_dir = os.getcwd()  # Fallback if __file__ is not defined (e.g., Jupyter)

# # Navigate to the directory containing utils.py (one level up)
# utils_dir = os.path.abspath(os.path.join(current_dir, '..'))

# # Add to sys.path
# sys.path.insert(0, utils_dir)

In [ ]:
# Now you can import utils
import utils
#### Create a sample AWS Lambda function that you want to convert into MCP tools
# This create role for lambda as well. This is the basic execution role
lambda_resp = utils.create_gateway_lambda("lambda_function_code.zip")
if lambda_resp is not None:
    if lambda_resp['exit_code'] == 0:
        print("Lambda function created with ARN: ", lambda_resp['lambda_function_arn'])
    else:
        print("Lambda function creation failed with message: ", lambda_resp['lambda_function_arn'])

Reading code from zip file
Creating IAM role for lambda function
Attaching policy to the IAM role
Role 'gateway_lambda_iamrole' created successfully: arn:aws:iam::381491838394:role/gateway_lambda_iamrole
Creating lambda function
Lambda function created with ARN:  arn:aws:lambda:ap-southeast-2:381491838394:function:gateway_lambda


In [ ]:
#### Create an IAM role for the Gateway to assume
import utils
# This role is able to run bedrock and invoke lambda
agentcore_gateway_iam_role = utils.create_agentcore_gateway_role("sample-lambdagateway")
print("Agentcore gateway role ARN: ", agentcore_gateway_iam_role['Role']['Arn'])

attaching role policy agentcore-sample-lambdagateway-role
Agentcore gateway role ARN:  arn:aws:iam::381491838394:role/agentcore-sample-lambdagateway-role


In [4]:
import time
import boto3
# CreateGateway with Amazon IAM. 
gateway_client = boto3.client('bedrock-agentcore-control', region_name = os.environ['AWS_DEFAULT_REGION'])

create_response = gateway_client.create_gateway(name='TestGWforLambdaIAM',
    roleArn = agentcore_gateway_iam_role['Role']['Arn'], # The IAM Role must have permissions to create/list/get/delete Gateway 
    protocolType='MCP',
    authorizerType='AWS_IAM',
    description='AgentCore Gateway with AWS Lambda target type using Amazon IAM for ingress auth'
)
print(create_response)
# Retrieve the GatewayID used for GatewayTarget creation
gatewayID = create_response["gatewayId"]
gatewayURL = create_response["gatewayUrl"]
print(gatewayID)
time.sleep(10)

{'ResponseMetadata': {'RequestId': '1f6379eb-3bc2-420b-aa87-1daba72c77b1', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Thu, 27 Nov 2025 22:59:15 GMT', 'content-type': 'application/json', 'content-length': '654', 'connection': 'keep-alive', 'x-amzn-requestid': '1f6379eb-3bc2-420b-aa87-1daba72c77b1', 'x-amzn-remapped-x-amzn-requestid': '59b8ca8c-4261-44b6-8b11-502d65bec4df', 'x-amzn-remapped-content-length': '654', 'x-amzn-remapped-connection': 'keep-alive', 'x-amz-apigw-id': 'UuamjF24SwMESKg=', 'x-amzn-trace-id': 'Root=1-6928d7c3-16e99846213f36e61cd818f7', 'x-amzn-remapped-date': 'Thu, 27 Nov 2025 22:59:15 GMT'}, 'RetryAttempts': 0}, 'gatewayArn': 'arn:aws:bedrock-agentcore:ap-southeast-2:381491838394:gateway/testgwforlambdaiam-o0ktjycpsr', 'gatewayId': 'testgwforlambdaiam-o0ktjycpsr', 'gatewayUrl': 'https://testgwforlambdaiam-o0ktjycpsr.gateway.bedrock-agentcore.ap-southeast-2.amazonaws.com/mcp', 'createdAt': datetime.datetime(2025, 11, 27, 22, 59, 15, 212184, tzinfo=tzutc()), 'upd

In [5]:
# Replace the AWS Lambda function ARN below
lambda_target_config = {
    "mcp": {
        "lambda": {
            "lambdaArn": lambda_resp['lambda_function_arn'], # Replace this with your AWS Lambda function ARN
            "toolSchema": {
                "inlinePayload": [
                    {
                        "name": "get_order_tool",
                        "description": "tool to get the order",
                        "inputSchema": {
                            "type": "object",
                            "properties": {
                                "orderId": {
                                    "type": "string"
                                }
                            },
                            "required": ["orderId"]
                        }
                    },                    
                    {
                        "name": "update_order_tool",
                        "description": "tool to update the orderId",
                        "inputSchema": {
                            "type": "object",
                            "properties": {
                                "orderId": {
                                    "type": "string"
                                }
                            },
                            "required": ["orderId"]
                        }
                    }
                ]
            }
        }
    }
}

credential_config = [ 
    {
        "credentialProviderType" : "GATEWAY_IAM_ROLE"
    }
]
targetname='LambdaUsingSDK'
response = gateway_client.create_gateway_target(
    gatewayIdentifier=gatewayID,
    name=targetname,
    description='Lambda Target using SDK',
    targetConfiguration=lambda_target_config,
    credentialProviderConfigurations=credential_config)

In [ ]:
#### Create an IAM role to Invoke Gateway
# this part is used to extract the user role and account info.
# the account number and gatewayID are used for the policy creation.
# The assume will be both for gateway and user role.
# in here Since I dont deploy the agent then my username will be the assumer of the role
current_role_arn = utils.get_current_role_arn()
print("Current role ARN: ", current_role_arn)

agentcore_gateway_iam_invoke_role = utils.create_gateway_invoke_tool_role("gateway-invoke-role",gatewayID , current_role_arn)
print("Role to invoke Agentcore gateway ARN: ", agentcore_gateway_iam_invoke_role['Role']['Arn'])

Current role ARN:  {'arn:aws:sts::381491838394:assumed-role/AWSReservedSSO_AWSAdministratorAccess_afee0d6ba2646b4f/PA.285833D@curtin.edu.au'}
Created new role: gateway-invoke-role
Attempt 1/5: AccessDenied, retrying in 3s...
Attempt 2/5: AccessDenied, retrying in 3s...
Attempt 3/5: AccessDenied, retrying in 3s...
Caller arn:aws:sts::381491838394:assumed-role/AWSReservedSSO_AWSAdministratorAccess_afee0d6ba2646b4f/PA.285833D@curtin.edu.au can now assume role gateway-invoke-role
 Role 'gateway-invoke-role' is ready and arn:aws:sts::381491838394:assumed-role/AWSReservedSSO_AWSAdministratorAccess_afee0d6ba2646b4f/PA.285833D@curtin.edu.au can invoke the Bedrock Agent Gateway.
Role to invoke Agentcore gateway ARN:  arn:aws:iam::381491838394:role/gateway-invoke-role


In [7]:
from strands.models import BedrockModel

## The IAM credentials configured in ~/.aws/credentials should have access to Bedrock model
yourmodel = BedrockModel(
    model_id="amazon.nova-pro-v1:0",
    temperature=0.7,
)

In [ ]:
from strands import Agent
import logging
from strands import Agent
import logging
from strands.tools.mcp.mcp_client import MCPClient
from mcp.client.streamable_http import streamablehttp_client 
from botocore.credentials import Credentials
from streamable_http_sigv4 import (
    streamablehttp_client_with_sigv4,
)

SERVICE="bedrock-agentcore"

# Configure the root strands logger. Change it to DEBUG if you are debugging the issue.
logging.getLogger("strands").setLevel(logging.INFO)

# Add a handler to see the logs
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()]
)

# This function is used in cognito not here
def create_streamable_http_transport(mcp_url: str, access_token: str):
       return streamablehttp_client(mcp_url, headers={"Authorization": f"Bearer {access_token}"})

def create_streamable_http_transport_sigv4(mcp_url: str, key: str, secret: str, sessionToken: str, serviceName: str, awsRegion: str):
        iamcredentials = Credentials(
            access_key=key,
            secret_key=secret,
            token = sessionToken
        )
        return streamablehttp_client_with_sigv4(
            url=mcp_url,
            credentials=iamcredentials,
            service=serviceName,
            region=awsRegion,
        )

def get_full_tools_list(client):
    more_tools = True
    tools = []
    pagination_token = None
    while more_tools:
        tmp_tools = client.list_tools_sync(pagination_token=pagination_token)
        tools.extend(tmp_tools)
        if tmp_tools.pagination_token is None:
            more_tools = False
        else:
            more_tools = True 
            pagination_token = tmp_tools.pagination_token
    return tools

def call_tool_sync(client, tool_id,tool_name, parameters=None):
    # Call the tool (no pagination argument supported)
    response = client.call_tool_sync(
        tool_use_id=tool_id,
        name=tool_name,
        arguments=parameters
    )

    # Extract output content
    if hasattr(response, "results") and response.results:
        return response.results
    elif hasattr(response, "output") and response.output:
        return response.output
    elif hasattr(response, "content"):
        return response.content
    else:
        return response  # fallback
         
def run_agent(mcp_url: str,key: str, secret: str, sessionToken: str,serviceName: str, awsRegion: str):
    mcp_client = MCPClient(lambda: create_streamable_http_transport_sigv4(mcp_url,key,secret,sessionToken,serviceName, awsRegion))

    with mcp_client:
        tools = get_full_tools_list(mcp_client)
        print(f"Found the following tools: {[tool.tool_name for tool in tools]}")
        print(f"Tool name: {tools[0].tool_name}")
        
        
        agent = Agent(model=yourmodel,tools=tools) ## you can replace with any model you like
        print(f"Tools loaded in the agent are {agent.tool_names}")
        agent("Check the order status for order id 123 and show me the exact response from the tool")
        #call mcp with tool
        tool=tools[0].tool_name
        tool_id="get-order-id-123-call-1"
        result = call_tool_sync(
            mcp_client,
            tool_id,
            tool_name=tool,
            parameters={"orderId": "123"}
        )

        print(f"Tool Call result: {result['content'][0]['text']}")

In [9]:
sts_client = boto3.client("sts")
response = sts_client.assume_role(
        RoleArn=agentcore_gateway_iam_invoke_role['Role']['Arn'],
        RoleSessionName="invoke_mcp_session",
        DurationSeconds=3600  # 1 hour, can be up to 12h for some roles
)

creds = response["Credentials"]

access=creds["AccessKeyId"]
secret=creds["SecretAccessKey"]
token=creds["SessionToken"]

# Run Agent with the credentials from this new gateway-invoke-role
time.sleep(10) 
run_agent(gatewayURL,access,secret,token, SERVICE, os.environ['AWS_DEFAULT_REGION'])

INFO | strands.telemetry.metrics | Creating Strands MetricsClient


Found the following tools: ['LambdaUsingSDK___get_order_tool', 'LambdaUsingSDK___update_order_tool']
Tool name: LambdaUsingSDK___get_order_tool
Tools loaded in the agent are ['LambdaUsingSDK___get_order_tool', 'LambdaUsingSDK___update_order_tool']
<thinking> To check the order status for order id 123, I need to use the `LambdaUsingSDK___get_order_tool` tool. This tool requires the `orderId` parameter, which I already have. </thinking>

Tool #1: LambdaUsingSDK___get_order_tool
The exact response from the tool is: "Order Id 123 is in shipped status". This indicates that the order with ID 123 has been shipped.Tool Call result: {"statusCode":200,"body":"Order Id 123 is in shipped status"}


Clean up

In [ ]:
import utils
utils.delete_gateway(gateway_client,gatewayID)